In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import cv2
import gym
# env = gym.make('ALE/Pong-v5', render_mode='rgb_array')
env = gym.make('PongDeterministic-v4', render_mode='rgb_array')


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(17, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.float()  # 将输入数据转换为float类型
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# 创建一个网络实例
net = Net()



In [5]:
def discretizer(state):
    greyimg = cv2.cvtColor(state[34:-16], cv2.COLOR_BGR2GRAY)
    unique_values = [123, 147, 236]
    coordinates = {}
    for value in unique_values:
        coordinates[value] = np.where(greyimg == value)
    #left center
    #print(coordinates[123], coordinates[147], coordinates[236])
    if coordinates[123][0].any():
        lc = (coordinates[123][0][0]+coordinates[123][0][-1])/2
    else:
        lc = -1
    #right center
    if coordinates[147][0].any():
        rc = (coordinates[147][0][0]+coordinates[147][0][-1])/2
    else:
        rc = -1
    #ball center
    if coordinates[236][0].any():
        bc = ((coordinates[236][0][0]+coordinates[236][0][-1])/2, (coordinates[236][1][0]+coordinates[236][1][-1])/2)
    else:
        bc = (-1,-1)
    
    return int(lc), int(rc), int(bc[0]),int(bc[1])

In [6]:
from collections import deque
import numpy as np
import pickle

# net = net.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
# 定义优化器
with open("net1000.pkl","rb") as f:
    net = pickle.load(f)
frame_no = 0
frames = deque(maxlen=16)
env.reset()
current_state, done = discretizer(env.reset()[0]), False
while not done:
    if frame_no < 4:
        action = env.action_space.sample()
        obs, reward, done,_, __ = env.step(action)
        for i in range(4):
            frames.append(current_state[i])
        frame_no += 1
        continue
    else:
        actions = [net(torch.tensor(list(frames)+[i])) for i in range(6)]
        action = actions.index(max(actions))
    for i in range(4):
        frames.append(torch.tensor(current_state[i])) 
    obs, reward, done, _,__ = env.step(action)
    current_state = discretizer(obs)
    frame_no += 1
    
    cv2.imshow('frame', np.asarray(obs))
    cv2.waitKey(1)